In [1]:
import glob
import cv2
from PIL import Image
import json
import os
from collections import OrderedDict

# Calcular dimensiones necesarias
Nuestro dataset proviene de dos fuentes: fotos propias y el dataset DETRAC.
Las primeras tiene un tamaño de 4608x2592, por lo tanto una relación de 4/3, mientras que DETRAC mide 960x540(16/9).

A la hora del entrenamiento todas las imágenes de entrada deben tener el mismo tamaño, así que vamos a reducir nuestras fotos al tamaño que utiliza DETRAC.

Primero empezamos calculando la nueva altura necesaria

In [2]:
DETRAC_width = 960
DETRAC_height = 540
DETRAC_relacion = DETRAC_width/DETRAC_height
print("DETRAC {}x{} = {}".format(DETRAC_width,DETRAC_height,DETRAC_relacion))

Alba_width = 4608
Alba_height = 3456
Alba_relacion = Alba_width/Alba_height
print("Alba {}x{} = {}".format(Alba_width,Alba_height,Alba_relacion))

Alba_width = 4608
Alba_height = Alba_width*DETRAC_height/DETRAC_width
Alba_relacion = Alba_width/Alba_height
print("Si recortamos la altura: Alba {}x{} = {}".format(Alba_width,Alba_height,Alba_relacion))

DETRAC 960x540 = 1.7777777777777777
Alba 4608x3456 = 1.3333333333333333
Si recortamos la altura: Alba 4608x2592.0 = 1.7777777777777777


# Pasar fotos a resolución 4608x2592
Resulta que la altura necesario para tener una resolución de 16/9 es de 2592 pixeles.

A continuación recortamos las fotos para que tengan dicho tamaño.

Utilizando la primera casilla podremos procesar las fotos donde los coches se vean cortados al cortarlos desde arriba de todo

In [ ]:
# Recortar una foto
img_path = "Fotos Alba/DSCF1466.JPG"
img_name = img_path.split("/")[-1]
im = Image.open(img_path)

recortada = im.crop((0, 850, 4608, 3442))
recortada.save("Fotos Alba recortadas/" + img_name[:-4] + ".jpg")

#Imagenes no recortadas desde (0,0)
#DSCF1445
#DSCF1455
#DSCF1461
#DSCF1463
#DSCF1466

In [ ]:
# Recortar toda la carpeta
for img_path in glob.glob("Fotos Alba/*.JPG"):
    print(img_path)
    img_name = img_path.split("/")[-1]
    im = Image.open(img_path)

    recortada = im.crop((0, 0, 4608, 2592))
    recortada.save("Fotos Alba recortadas/" + img_name[:-4] + ".jpg")


## Pasar anotaciones de 4608 × 2598 a 960×540

A continuación reducimos la resolución de las imágenes

In [ ]:
# Recortar toda la carpeta
for img_path in glob.glob("Fotos Alba recortadas/*.jpg"):
    print(img_path)
    img_name = img_path.split("/")[-1]
    im = Image.open(img_path)

    size = 960, 540
    recortada = im.resize(size, Image.ANTIALIAS)
    recortada.save("Fotos Alba resolucion OK/" + img_name[:-4] + ".jpg")

# Pasar anotaciones a nueva resolución
Por último, como las imágenes ya estaban anotadas debemos de pasar las anotaciones a las nuevas medidas

In [ ]:
### Procesar una imagen
# Leer archivo de anotaciones
data = json.load(open('Fotos Alba/via_region_data (1).json'))
init_width = 4608
init_height = 3456
final_width = 960
final_height = 720

# Calculo la clave
image_name = 'DSCF1440.JPG'
image_path = 'Fotos Alba/' + image_name
image_size = os.stat(image_path).st_size # Tamanho
image_name = os.path.basename(image_path)    # Nombre
vgg_key = "{}{}".format(image_name, image_size)
datos_leidos = data[vgg_key]
print(data[vgg_key])

# Añado al json
new_image_path = 'Fotos Alba resolucion OK/' + image_name
new_image_size = os.stat(new_image_path).st_size # Tamanho
new_image_name = os.path.basename(new_image_path)[:-4] + ".jpg"    # Nombre
data[vgg_key]['size'] = new_image_size
data[vgg_key]['filename'] = new_image_name
print(data[vgg_key])

anotaciones = data[vgg_key]['regions']
for anotacion in anotaciones:
    print(anotaciones[anotacion]['shape_attributes'])
    new_x = final_width * anotaciones[anotacion]['shape_attributes']['x'] / init_width
    new_y = final_height * anotaciones[anotacion]['shape_attributes']['y'] / init_height
    new_width = final_width * anotaciones[anotacion]['shape_attributes']['width'] / init_width
    new_height = final_height* anotaciones[anotacion]['shape_attributes']['height'] / init_height
    
    anotaciones[anotacion]['shape_attributes']['x'] = new_x
    anotaciones[anotacion]['shape_attributes']['y'] = new_y
    anotaciones[anotacion]['shape_attributes']['width'] = new_width
    anotaciones[anotacion]['shape_attributes']['height'] = new_height
    
print(data[vgg_key])

new_key = new_image_name + str(new_image_size)
vgg_main = {new_key : data[vgg_key]}

with open("prueba.json", "w") as f:
            json.dump(vgg_main, f)
    

In [ ]:
# Procesar una carpeta

# Leer archivo de anotaciones
data = json.load(open('Fotos Alba/via_region_data (1).json'))
init_width = 4608
init_height = 3456
final_width = 960
final_height = 720
vgg_main = OrderedDict()

for image_path in glob.glob("Fotos Alba/*.JPG"):

    # Calculo la clave
    print(image_path)
    image_name = image_path.split("/")[-1]
    image_size = os.stat(image_path).st_size # Tamanho
    image_name = os.path.basename(image_path)    # Nombre
    vgg_key = "{}{}".format(image_name, image_size)
    datos_leidos = data[vgg_key]
    #print(data[vgg_key])

    # Añado al json
    new_image_path = 'Fotos Alba resolucion OK/' + image_name
    new_image_size = os.stat(new_image_path).st_size # Tamanho
    new_image_name = os.path.basename(new_image_path)[:-4] + ".jpg"    # Nombre
    data[vgg_key]['size'] = new_image_size
    data[vgg_key]['filename'] = new_image_name
    #print(data[vgg_key])

    anotaciones = data[vgg_key]['regions']
    for anotacion in anotaciones:
        #print(anotaciones[anotacion]['shape_attributes'])
        new_x = final_width * anotaciones[anotacion]['shape_attributes']['x'] / init_width
        new_y = final_height * anotaciones[anotacion]['shape_attributes']['y'] / init_height
        new_width = final_width * anotaciones[anotacion]['shape_attributes']['width'] / init_width
        new_height = final_height* anotaciones[anotacion]['shape_attributes']['height'] / init_height

        anotaciones[anotacion]['shape_attributes']['x'] = new_x
        anotaciones[anotacion]['shape_attributes']['y'] = new_y
        anotaciones[anotacion]['shape_attributes']['width'] = new_width
        anotaciones[anotacion]['shape_attributes']['height'] = new_height
        anotaciones[anotacion]['shape_attributes']['height'] = new_height
        anotaciones[anotacion]["region_attributes"]["Class"] = "car"

    #print(data[vgg_key])

    new_key = new_image_name + str(new_image_size)
    vgg_main.update({new_key : data[vgg_key]})

with open("prueba.json", "w") as f:
    json.dump(vgg_main, f)
